<a href="https://colab.research.google.com/github/efratrvd/tel-aviv-rent-analysis/blob/main/gmaps_nearby_places.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prettyprint
!pip install googlemaps

In [ ]:
import pandas as pd
import googlemaps
import pprint
import json
import time
import os


Get near_by place data from google for all locations

In [ ]:
def get_near_by_data_for_location(row):
  ID, lati, longi = row
  prop_name = ID 
  result_dir = "/content/drive/MyDrive/BGU/BIGDATA/"
  result_list = []
  next_page_token = None
  for i in range(1):
    if i == 0:
      place_result = gmaps.places_nearby(location='{},{}'.format(lati, longi), radius=300, open_now = False)
    else:
      place_result = gmaps.places_nearby(page_token=next_page_token)
    next_page_token = place_result.get('next_page_token', None)
    results = place_result.get('results', [])
    result_list.extend(results)
    time.sleep(3)
    if next_page_token is None:
      break
  
  with open(result_dir+prop_name+'.json', 'w', encoding='utf-8') as f:
      json.dump(result_list, f, ensure_ascii=False, indent=4)

In [ ]:
#  api key here
gmaps = googlemaps.Client(key = api)

all_data_df = pd.read_csv("/content/drive/MyDrive/ta-rent/properties3.csv", usecols=['_id','address.street', 'squareMeters','rentPrice','areas','floor','coords.lat','coords.lng'])
df = all_data_df.dropna(subset=['address.street', 'squareMeters','rentPrice','areas','floor','coords.lat','coords.lng'])

In [ ]:
df[['_id', 'coords.lat','coords.lng']].apply(get_near_by_data_for_location, axis=1)

Create dataset of near by places

In [ ]:
result_dir = "/content/drive/MyDrive/BGU/BIGDATA/"
all_prop_data = []
for dir, folders, files in os.walk(result_dir):
  for file in files:
    prop_data = {'_id': str(file).replace('.json',''), 'places_name': [], 'places_rating': [], 'places_types': [], 'places_num_rating': []}
    with open(os.path.join(dir,file)) as j:
      gmap_json = json.loads(j.read())
      for i, item in enumerate(gmap_json):
        if i == 0:
          continue
        prop_data['places_name'].append(item['name'])
        prop_data['places_rating'].append(item.get('rating', None))
        prop_data['places_types'].append(item['types'])
        prop_data['places_num_rating'].append(item.get('user_ratings_total', None))
      all_prop_data.append(prop_data)
property_nearby_places_df = pd.DataFrame(all_prop_data)

In [ ]:
property_nearby_places_df.to_csv("/content/drive/MyDrive/BGU/property_nearby_places_df.csv")